In [1]:
from preprocessing.preprocessing import image_scaling, convert_image, label_conversion
import torch
import torchinfo
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
import os
import json
import gc
from PIL import Image
import numpy as np
import timm
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

c:\Users\eric_\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
image_folder = 'images'
BATCH_SIZE = 32
# Load calorie database (for food labels)
CALORIE_DB_FILE = os.path.join('preprocessing', "calories_database.json")
with open(CALORIE_DB_FILE, "r") as f:
    calorie_db = json.load(f)

FOOD_LABELS = sorted(list(calorie_db.keys()))
NUM_CLASSES = len(FOOD_LABELS)


In [4]:
# Define Custom Dataset
class FoodPortionDataset(Dataset):
    def __init__(self, json_path, img_dir, transform=None):
        with open(json_path, "r") as f:
            self.data = json.load(f)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img_path = os.path.join(self.img_dir, item["name"] + ".png")
        image = Image.open(img_path).convert("RGB")

        # Convert food types to a one-hot encoded vector
        food_vector = torch.zeros(NUM_CLASSES)
        portion_vector = torch.zeros(NUM_CLASSES)

        for food, portion in zip(item["food type"], item["portion"]):
            if food in FOOD_LABELS:
                food_idx = FOOD_LABELS.index(food)
                food_vector[food_idx] = 1  # Detected food category
                portion_vector[food_idx] = float(portion)  # Ground truth portion size

        if self.transform:
            image = self.transform(image)

        return image, food_vector, portion_vector  # Return image, detected foods, and portion sizes

In [ ]:
# Define Data Transforms
transform = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load Datasets
train_dataset = FoodPortionDataset(json_path=os.path.join('preprocessing', "train.json"), 
                            img_dir="images_resized",
                            transform=transform)
test_dataset = FoodPortionDataset(json_path=os.path.join('preprocessing', "test.json"), 
                          img_dir="images_resized",
                          transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, model, num_embed, num_classes):
        super(CustomModel, self).__init__()
        self.model = model
        # self.model.fc = nn.Identity()
        self.fc_food = nn.Linear(num_classes, 64)
        self.fc_combine = nn.Sequential(
            nn.Linear(num_embed + 64, 128),  # Image features + food class vector
            # nn.ReLU(),
            # nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(128, num_classes),  # Predict portion sizes for each food type
            nn.ReLU()
        )
    
    def forward(self, img, food_vec):
        model_out = self.model(img)
        food_vec_embed = self.fc_food(food_vec)
        concat = torch.cat((model_out, food_vec_embed), dim=1)
        return self.fc_combine(concat)

In [ ]:
def train(model, trainloader, optimizer, criterion, device, epochs=10):
    min_loss = np.inf
    for epoch in range(epochs):
        running_loss = 0
        model.train()
        for images, labels, portions in trainloader:
            images, labels, portions = images.to(device), labels.to(device).float(), portions.to(device).float()
            optimizer.zero_grad()
            out = model(images, labels)
            loss = criterion(out, portions)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # delete from memory
            del images
            del labels
            torch.cuda.empty_cache()
            gc.collect()
        loss = running_loss/len(train_loader)
        if loss < min_loss: min_loss = loss
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")
    return min_loss

In [8]:
def evaluate(model, testloader, device):
    model.eval() # Set the model to evaluation mode
    mse_loss = 0
    criterion = nn.MSELoss()
    total_size = 0
    with torch.no_grad():
        for images, labels, portions in testloader:
            images, labels, portions = images.to(device), labels.to(device).float(), portions.to(device).float()
            outputs = model(images, labels)
            print(f"real out: {portions}")
            print(f"model out: {outputs}")
            loss = criterion(outputs, portions)
            mse_loss += loss.item() * images.size(0)  # batch size
            total_size += images.size(0)

            del images
            del labels
            torch.cuda.empty_cache()
            gc.collect()

    return mse_loss / total_size

In [9]:
results = []

img_features_list = [16,32,64,128,256,512]

for img_features in img_features_list:
    resnet = timm.create_model('resnet34', pretrained=True, num_classes=img_features)
    model = CustomModel(resnet, img_features, NUM_CLASSES).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    print(f"Training model with CNN image features {img_features}")
    train_loss = train(model, train_loader, optimizer, criterion, device, epochs=20)
    print("Training complete")
    avg_loss = evaluate(model, test_loader, device)
    print(f"{img_features} Average Loss: {avg_loss}")
    results.append({
        'model embed features': img_features,
        'best training loss': train_loss,
        'test avg loss': avg_loss
    })
    del model
    torch.cuda.empty_cache()
    gc.collect()

results_df = pd.DataFrame(results)
results_df

Training model with CNN image features 16
Epoch 1/20, Loss: 702.4278970445905
Epoch 2/20, Loss: 572.2639825003488
Epoch 3/20, Loss: 538.8973759242466
Epoch 4/20, Loss: 541.4903346470425
Epoch 5/20, Loss: 525.7568566458566
Epoch 6/20, Loss: 497.2531547546387
Epoch 7/20, Loss: 493.57706614903043
Epoch 8/20, Loss: 484.52736173357283
Epoch 9/20, Loss: 448.7962074279785
Epoch 10/20, Loss: 430.9291283743722
Epoch 11/20, Loss: 414.26507295881
Epoch 12/20, Loss: 367.1849411555699
Epoch 13/20, Loss: 317.5411594935826
Epoch 14/20, Loss: 307.8403284890311
Epoch 15/20, Loss: 271.6621924809047
Epoch 16/20, Loss: 279.69693401881625
Epoch 17/20, Loss: 264.1512284960066
Epoch 18/20, Loss: 246.21533993312292
Epoch 19/20, Loss: 247.76708602905273
Epoch 20/20, Loss: 233.073796408517
Training complete
real out: tensor([[  0.,   0.,   0.,   0.,  38.,   0.,   0.,  35.,   0.,   0.,   5.,  35.,
           0.,   0.,   0.,   0.,  16.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.],
        [  0.

,model embed features,best training loss,test avg loss
0,16,233.073796,206.018966
1,32,237.078241,215.234368
2,64,233.885111,223.395914
3,128,276.942113,287.771291
4,256,250.650060,256.916882
5,512,222.294802,224.905237
